In [1]:
# Ce notebook contient les mêmes méthodes que le premier mais executées sur les features extraites par Mr. Hanna
# contrairement à l'autre qui repose sur notre propre extraction avec librosa

# Executez cette case si vous n'avez pas les données dans le répertoire

# !wget http://dept-info.labri.fr/~hanna/Pub/features_adapte.csv
# !wget http://dept-info.labri.fr/~hanna/Pub/features_head.csv
# !wget http://dept-info.labri.fr/~hanna/Pub/train_clean.csv
# !wget http://dept-info.labri.fr/~hanna/Pub/test_clean.csv

--2020-01-28 15:03:19--  http://dept-info.labri.fr/~hanna/Pub/features_adapte.csv
Résolution de dept-info.labri.fr (dept-info.labri.fr)… 147.210.9.83, 2001:660:6101:404::81
Connexion à dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 951108530 (907M) [text/csv]
Enregistre : «features_adapte.csv»

features_adapte.csv 100%[===================>] 907,05M  9,04MB/s    ds 1m 41s  

2020-01-28 15:05:00 (8,99 MB/s) - «features_adapte.csv» enregistré [951108530/951108530]

--2020-01-28 15:05:00--  http://dept-info.labri.fr/~hanna/Pub/features_head.csv
Résolution de dept-info.labri.fr (dept-info.labri.fr)… 147.210.9.83, 2001:660:6101:404::81
Connexion à dept-info.labri.fr (dept-info.labri.fr)|147.210.9.83|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 19134 (19K) [text/csv]
Enregistre : «features_head.csv»

features_head.csv   100%[===================>]  18,69K  --.-KB/s    ds 0

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
# Croisement features/tracks du dataset train

# Nom des features
features = pd.read_csv(filepath_or_buffer="features_head.csv", sep=",")
print(features.columns)
#print(features)
print("#################")

# Croisement features/tracks du dataset train
traingenre = pd.read_csv(filepath_or_buffer="train_clean.csv", sep=",")
iter_csv = pd.read_csv(filepath_or_buffer="features_adapte.csv", sep=",", iterator=True, chunksize=10000)
datatrain = pd.concat([chunk for chunk in iter_csv])

data = pd.merge(traingenre, datatrain, on='track_id')
data.sample(n=10)

Index(['feature', 'chroma_cens', 'chroma_cens.1', 'chroma_cens.2',
       'chroma_cens.3', 'chroma_cens.4', 'chroma_cens.5', 'chroma_cens.6',
       'chroma_cens.7', 'chroma_cens.8',
       ...
       'tonnetz.39', 'tonnetz.40', 'tonnetz.41', 'zcr', 'zcr.1', 'zcr.2',
       'zcr.3', 'zcr.4', 'zcr.5', 'zcr.6'],
      dtype='object', length=519)
#################


,track_id,genre_id,01,02,03,04,05,06,07,08,...,04.41,05.41,06.41,01.70,01.71,01.72,01.73,01.74,01.75,01.76
3544,133294,6,0.133676,1.219621,1.041329,0.659568,0.661551,0.840704,1.389098,1.094413,...,0.063345,0.016298,0.018405,3.186660,0.575195,0.092824,0.066406,0.000977,1.689548,0.080830
1023,40237,3,-0.634775,-0.452372,-0.081961,0.020517,-0.575869,-0.431036,-0.426676,-0.382549,...,0.097534,0.017034,0.017351,16.264420,0.691406,0.073199,0.055176,0.009766,2.996620,0.062984
3698,140872,3,-1.081832,-1.001203,0.902922,0.164892,-0.614164,-0.655986,-0.119395,5.273300,...,0.152206,0.030529,0.025080,30.582483,0.436523,0.025041,0.014160,0.004883,5.332044,0.046589
272,11419,1,-0.768838,-0.667095,-0.604739,-0.180145,-0.846874,-0.380538,-0.984623,-0.918730,...,0.140241,0.036428,0.028642,35.297848,0.337891,0.030125,0.024414,0.000000,5.388410,0.027061
620,24331,6,-0.504906,-0.532965,-0.224551,-0.419990,-0.410279,0.823258,0.188249,-0.768032,...,0.168203,0.034010,0.031814,295.583923,0.840332,0.021472,0.014648,0.000000,15.710932,0.040361
1950,69198,2,0.750938,0.909248,4.442852,2.286593,1.375715,2.297371,-0.726958,-0.612376,...,0.090088,0.015022,0.018651,26.359732,0.840820,0.054976,0.031738,0.000000,4.328013,0.079193
3765,143979,4,4.442158,10.669847,3.940961,1.567164,0.490692,-0.452973,-0.212948,0.164879,...,0.095396,0.019743,0.021407,4.454741,0.453613,0.077846,0.062988,0.001465,1.859602,0.066624
1310,48820,3,-0.517542,0.001574,-0.178640,-0.108170,-0.252360,0.278714,-0.854324,0.411887,...,0.093371,0.025141,0.020462,6.162170,0.284180,0.048311,0.035645,0.000000,2.079569,0.039280
3769,144175,4,-0.390341,0.436722,-0.609044,-0.538162,0.086064,-0.741646,-0.939368,-0.657182,...,0.113712,0.026862,0.034028,40.887131,0.743652,0.050757,0.041016,0.000000,4.673950,0.046759
2511,91937,3,2.130329,-0.097011,0.293347,-0.371048,1.435453,1.242170,1.015294,1.811976,...,0.097805,0.021688,0.027949,56.397892,0.331543,0.024451,0.020508,0.000000,5.723203,0.019159


In [4]:
# training sets
x = data.drop(['genre_id', 'track_id'], axis=1)
y = data['genre_id'].values

print("x :", x.shape, ", y :", y.shape)

x : (3995, 518) , y : (3995,)


In [5]:
# split train test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

print("x_train :", x_train.shape, ", y_train :", y_train.shape)
print("x_test :", x_test.shape, ", y_test :", y_test.shape)

x_train : (3196, 518) , y_train : (3196,)
x_test : (799, 518) , y_test : (799,)


In [6]:
# normalisation
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

mean_train = x_train.mean()
std_train = x_train.std()
print(f'mean_train: {mean_train}. std_train: {std_train}')

mean_train: 7.289002228542061e-17. std_train: 0.996131513545271


## Premier modèle : K nearest neighbors

In [7]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [8]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.82      0.90      0.86        88
           2       0.54      0.36      0.43       105
           3       0.63      0.50      0.56       109
           4       0.57      0.52      0.54       110
           5       0.37      0.69      0.49        97
           6       0.46      0.22      0.30       100
           7       0.27      0.33      0.30        95
           8       0.45      0.51      0.48        95

    accuracy                           0.50       799
   macro avg       0.52      0.50      0.49       799
weighted avg       0.52      0.50      0.49       799



## Deuxième modèle : Decision Tree

In [9]:
tree_model = DecisionTreeClassifier(max_leaf_nodes=16, min_samples_split=2)
tree_model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [10]:
y_pred = tree_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.78      0.65      0.71        88
           2       0.34      0.17      0.23       105
           3       0.44      0.63      0.52       109
           4       0.44      0.49      0.46       110
           5       0.60      0.22      0.32        97
           6       0.33      0.21      0.26       100
           7       0.18      0.40      0.25        95
           8       0.53      0.48      0.51        95

    accuracy                           0.41       799
   macro avg       0.46      0.41      0.41       799
weighted avg       0.45      0.41      0.40       799



## Troisième modèle : Random Forest

In [11]:
forest_model = RandomForestClassifier(max_leaf_nodes=16, min_samples_split=2)
forest_model.fit(x_train, y_train)

/home/janerussel/anaconda3/envs/IA/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=16,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [12]:
y_pred = forest_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.69      0.84      0.76        88
           2       0.35      0.27      0.30       105
           3       0.50      0.66      0.57       109
           4       0.47      0.62      0.54       110
           5       0.39      0.37      0.38        97
           6       0.35      0.22      0.27       100
           7       0.35      0.14      0.20        95
           8       0.52      0.73      0.61        95

    accuracy                           0.48       799
   macro avg       0.45      0.48      0.45       799
weighted avg       0.45      0.48      0.45       799



## Quatrième méthode : Gradient Boosting

In [13]:
gradient_model = GradientBoostingClassifier()
gradient_model.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [14]:
y_pred = gradient_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.87      0.86      0.87        88
           2       0.52      0.42      0.47       105
           3       0.68      0.64      0.66       109
           4       0.57      0.60      0.58       110
           5       0.59      0.63      0.61        97
           6       0.55      0.48      0.51       100
           7       0.26      0.34      0.30        95
           8       0.66      0.67      0.67        95

    accuracy                           0.58       799
   macro avg       0.59      0.58      0.58       799
weighted avg       0.59      0.58      0.58       799



## Cinquième méthode : XGBoost

In [15]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [16]:
y_pred = xgb_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.88      0.91      0.89        88
           2       0.52      0.50      0.51       105
           3       0.66      0.72      0.69       109
           4       0.57      0.63      0.60       110
           5       0.59      0.56      0.57        97
           6       0.56      0.45      0.50       100
           7       0.31      0.32      0.31        95
           8       0.60      0.63      0.62        95

    accuracy                           0.59       799
   macro avg       0.59      0.59      0.59       799
weighted avg       0.59      0.59      0.59       799



## Tuning XGBoost

In [17]:
xgb_model = xgb.XGBClassifier(n_estimators=200, learning_rate=0.2, max_depth=3, min_child_weight=1)
xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [18]:
y_pred = xgb_model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.88      0.92      0.90        88
           2       0.54      0.52      0.53       105
           3       0.69      0.67      0.68       109
           4       0.66      0.66      0.66       110
           5       0.66      0.67      0.66        97
           6       0.54      0.48      0.51       100
           7       0.40      0.42      0.41        95
           8       0.63      0.66      0.65        95

    accuracy                           0.62       799
   macro avg       0.62      0.63      0.62       799
weighted avg       0.62      0.62      0.62       799

